In [13]:
from bs4 import BeautifulSoup
import requests
import re
import pprint
import wordfreq
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import pandas as pd
import random
import datetime
from sklearn.cluster import SpectralClustering

These functions are meant to extract presidential speeches from the site millercenter.org.  get_president(N) finds the Nth president and returns a "speeches" variable that is a list containing the text of each speech by that president.  The president's number is 1-indexed, so George Washington is president number 1.  This convention is followed throughout, except where otherwise specified.

In [14]:
#N = 8396
N = 1

# The 1st president (Washington) is actually numbered 44 on millercenter.org.  Thus, we have to convert
# the president's number 1 through 44 into a number suitable for millercenter.org.  Trump is, randomly enough
# president number 8396.
# The 22nd and 24th president are ordinarily considered to be Cleveland, Cleveland is only counted once.
# #24 is considered to be Harrison; that means there are only 44 presidents
def pres_numbers_list():
    return [44, 45] + [3, 4] + [141] + list(range(6, 44)) + [8396]

def get_pres_name(N):
    return ["George Washington", "John Adams", "Thomas Jefferson", "James Madison", "James Monroe", "John Quincy Adams",\
           "Andrew Jackson", "Martin van Buren", "William Harrison", "John Tyler", "James K. Polk", "Zachary Taylor",\
           "Millard Fillmore", "Franklin Pierce", "James Buchanan", "Abraham Lincoln", "Andrew Johnson",\
           "Ulysses S. Grant", "Rutherford B. Hayes", "James A. Garfield", "Chester A. Arthur", "Grover Cleveland",\
           "Benjamin Harrison", "William McKinley", "Theodore Roosevelt", "William Taft", "Woodrow Wilson",\
           "Warren G. Harding", "Calvin Coolidge", "Herbert Hoover", "Franklin D. Roosevelt", "Harry S. Truman",\
           "Dwight D. Eisenhower", "John F. Kennedy", "Lyndon B. Johnson", "Richard Nixon", "Gerald Ford",\
           "Jimmy Carter", "Ronald Reagan", "George H. W. Bush", "Bill Clinton", "George W. Bush", "Barack Obama",\
           "Donald Trump"][N-1]

def get_president(N):
    if N < 1 or N > 44:
        print("Error: no president", N)
        return list()
    N = pres_numbers_list()[N - 1]
    millerpage = f"https://millercenter.org/the-presidency/presidential-speeches?field_president_target_id[{N}]={N}"
    page = requests.get(millerpage)
    soup = BeautifulSoup(page.content, 'html.parser')
    #dummy = 'a href="/the-presidency/presidential-speeches'
    speechlist = soup.find_all(href=re.compile('/the-presidency/presidential-speeches/'))
    URLlist = ["https://millercenter.org" + x['href'] for x in speechlist]
    #pprint.pprint(URLlist)
    speeches = list()
    n = -1
    for URL in URLlist:
        n = n + 1
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        for e in soup.find_all('br'):
            e.replace_with('\n')
        x = soup('h3', text="Transcript")
        listofps = x[0].parent.findChildren('p')
        #print([type(p.contents[0]) for p in listofps])
        #string = string.replace(u'\xa0', u' ')
        # print([p.contents[0].name for p in listofps])
        textofspeech = " ".join([" ".join([c for c in p.contents if not c.name]) for p in listofps])
        # Replace funny single quote #8217 with single quote #39
        textofspeech = textofspeech.replace('\’', "'")
        textofspeech = textofspeech.replace("\'", "'")
        speeches.append(textofspeech)
    return speeches


get_overrep_words takes a list of the text of each speech, and returns a list of "overrepresented words" the form:

[(word1, weight1), (word2, weight2), ...]

The "weight" for each word is calculated as follows.  The goal is to have a weight that reflects (1) the relative incidence of the word in the speech corpus compared with its usual incidence in the English language, (2) the overall incidence of the word in the speech corpus.  Thus, a word that is highly unusual ("supercalifragilisticexpealidocious") but only appears once in the speech corpus is not that interesting.  In contrast, a word that is very common, but appears only the usual amount in the corpus ("the") is also not interesting.  The weight for each words is computed as a product of these two factors.

Getting into the details: To compute the weight, the incidence count of the word is first tallied in the combined speeches of a president.
Using the wordfreq library, each word's frequency in English is obtained (actually the log of the frequency), so that it can be compared with the log of the frequency in the speeches.
Using the statsmodels library, a least squares regression is used to find the linear relationship between these two logs.

This linear relationship can then be used to compute the predicted English frequency for a word that has the given corpus frequency.  This predicted English frequency can be compared to the word's actual English frequency.  The difference between predicted and actual (using subtraction) is essentially a measure of how much more common the word is in the speech corpus compared to the English language.  A weight value of 1 would mean that the word is "e" times more common in the speech corpus as compared with English.

Next, the number of words in each speech can range from 20,000 to 800,000, so the number of appearances is multiplied by (800,000 / length) to get the number that would appear in 800,000.  Then, the log is taken.  This number is multiplied by the weight to get the final weight value.  This takes into account that we care more about words that appear many times as opposed to only once.

In [15]:
def get_overrep_words(speeches):
    speeches_combined = " ".join(speeches)
    print("Length of speeches:", len(speeches_combined))
    wordlist = wordfreq.tokenize(speeches_combined, 'en')
    length = len(wordlist)
    freqlist = [(x, wordlist.count(x)) for x in set(wordlist)]
    freqlist.sort(key = lambda x: x[1])
    freqmap = [(np.log(wordfreq.word_frequency(x[0], 'en')), np.log(x[1] / length)) for x in freqlist]
    xy = list(zip(*freqmap))
    #plt.figure()
    #plt.plot(*xy, 'b.')
    dfxy = pd.DataFrame(xy).T
    dfxy.columns = ['logen', 'logcorpus']
    dfxy = dfxy.replace([np.inf, -np.inf], np.nan).dropna()
    #print(dfxy)
    model = sm.OLS.from_formula('logen ~ logcorpus', dfxy) 
    regr = model.fit()
    #regr.summary()

    # Replace funny single quote #8217 with single quote #39
    freqlist = [(x[0].replace('’', "'"), x[1]) for x in freqlist]
    
    weightlist = [(x[0], (regr.predict(pd.Series([np.log(x[1] / length)], name='logcorpus')).iloc[0] - \
            np.log(wordfreq.word_frequency(x[0], 'en'))) * np.log(x[1] * 800000 / length) \
            ) for x in freqlist if ((x[1] * 800000 / length > 1) and x[0] != 'applause' and x[0] != 'laughter')]
    
    weightlist.sort(key = lambda x: x[1])

    return [x for x in weightlist if (x[1] > 1 and not np.isinf(x[1]))], freqlist

In [16]:
def make_two_speech_lists(speeches):
    lst1 = np.random.choice(speeches, size=int(len(speeches)/2), replace=False)
    lst2 = [x for x in speeches if not (x in lst1)]
    return (lst1, lst2)

get_paired_words returns a list of the form:

[(word1, sim1), (word2, sim2), ...]

Where the similarity score (sim1) for word 1 is the minimum of the word's weight for president M and for president N.  Only words that appear in both president M and president N's speeches will get a score.

get_similarity finds the total similarity score by adding them up.

The result is a similarity score that compares the two presidents.  If the presidents share more words in common (and with higher weights), then their similarity score will tend to be higher.

In [38]:
def get_paired_words(overrep_words, M, N):
    if M < 1 or M > len(overrep_words) or N < 1 or N > len(overrep_words):
        print("M or N is out of bounds in get_paired_words.")
        return list()
    paired_words = list()
    for x in overrep_words[M-1]:
        for y in overrep_words[N-1]:
            if x[0] == y[0]:
                paired_words.append((x[0], min(x[1],y[1])))
    return paired_words

def get_similarity(overrep_words, M, N):
    pw = get_paired_words(overrep_words, M, N)
    total = sum(x[1] for x in pw)
    return total

This code block loops through a range of presidents, getting a list of overrepresented words (with their weights) and a list of frequencies for each.

In [20]:
overrep_words = list()
overrep_words_test = list()

NUMP = 44

train_test = True

random.seed(datetime.datetime.now())

for N in range(45 - NUMP, 45):
    print(get_pres_name(N))
    speeches = get_president(N)
    sp_train, sp_test = make_two_speech_lists(speeches)
    if train_test:
        if len(sp_train) == 0:
            sp_train = sp_test.copy()
            print("Only one (testing) speech for president ", N)
        if len(sp_test) == 0:
            sp_test = sp_train.copy()
            print("Only one (training) speech for president ", N)
        ov, fr = get_overrep_words(sp_train)
        ov_test, fr_test = get_overrep_words(sp_test)
        overrep_words.append(ov)
        overrep_words_test.append(ov_test)
    else:
        ov, fr = get_overrep_words(speeches)
        overrep_words.append(ov)

George Washington
Length of speeches: 77699


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


Length of speeches: 50943
John Adams
Length of speeches: 32447
Length of speeches: 56462
Thomas Jefferson
Length of speeches: 62251
Length of speeches: 54366
James Madison
Length of speeches: 83068
Length of speeches: 53755
James Monroe
Length of speeches: 150449
Length of speeches: 146772
John Quincy Adams
Length of speeches: 140387
Length of speeches: 81486
Andrew Jackson
Length of speeches: 302763
Length of speeches: 205487
Martin van Buren
Length of speeches: 153268
Length of speeches: 237930
William Harrison
Only one (testing) speech for president  9
Length of speeches: 49777
Length of speeches: 49777
John Tyler
Length of speeches: 101705
Length of speeches: 175165
James K. Polk
Length of speeches: 275449
Length of speeches: 50042
Zachary Taylor
Length of speeches: 56131
Length of speeches: 12422
Millard Fillmore
Length of speeches: 148159
Length of speeches: 87095
Franklin Pierce
Length of speeches: 141813
Length of speeches: 164628
James Buchanan
Length of speeches: 135614
Lengt

Create a similarity matrix such that X[i,j] is the similarity of presidents i and j.  Then, print the cluster labels and cluster centers.

In [39]:
import random
from datetime import datetime

def similarity_matrix(overrep_words):
    X = np.zeros((NUMP, NUMP))
    for i in range(0, NUMP):
        for j in range(0, NUMP):
            X[i,j] = get_similarity(overrep_words, i+1, j+1)
    return X

n_clusters = 3

X = similarity_matrix(overrep_words)
#clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity="precomputed", linkage="average").fit(X)
clustering = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", n_neighbors=10, assign_labels="discretize").fit(X)
print("Labels:", clustering.labels_)
#print("Centers:", clustering.cluster_centers_)

if train_test:
    X_test = similarity_matrix(overrep_words_test)
    #clustering_test = AgglomerativeClustering(n_clusters=n_clusters, affinity="precomputed", linkage="average").fit(X_test)    
    clustering_test = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", n_neighbors=10, assign_labels="discretize").fit(X_test)
    print("Labels test:", clustering_test.labels_)

Labels: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]
Labels test: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 2 2 2 2 2 2 2
 2 2 2 2 2 2 2]
Num possible pairs:  343
Num similar pairs:  263


computer_cluster_overlap will compute the overlap between the training and testing data.  The overlap is defined by assuming that each cluster in the training data corresponds to the cluster(s) in the testing data with the best fit (the maximum overlap.)  Then, the number of presidents whose training cluster matches their testing cluster is counted.

In [51]:
def compute_cluster_overlap(labels1, labels2):
    if len(labels1) != len(labels2):
        print("Error in compute_cluster_overlap: labels1, labels2 not same length.")
        return
    corresp = dict()
    for lab1 in set(labels1):
        ct = dict()
        for lab2 in set(labels2):
            for i in range(len(labels1)):
                ct.setdefault(lab2, 0)
                if labels1[i] == lab1 and labels2[i] == lab2:
                    ct[lab2] += 1
        ctreverse={v:k for k,v in ct.items()}
        corresp[lab1] = ctreverse[max(ctreverse)]
    total = 0
    for i in range(len(labels1)):
        if labels2[i] == corresp[labels1[i]]:
            total += 1
    print("Correspondence (training cluster: testing cluster):", corresp)
    print("Overlap ratio:", total, "out of", len(labels1))        
    return total, len(labels1)

compute_cluster_overlap(clustering.labels_, clustering_test.labels_)

Correspondence (training cluster: testing cluster): {0: 0, 1: 2, 2: 1}
Overlap ratio: 41 out of 44


(41, 44)

These functions make use of only the training set data (or all of the data, if you have set train_test = False).

get_inlist will make a list of the numbers of the president that belong to the ith cluster in the training set (zero indexed this time.)

get_characteristic_words accepts as an argument a list of president numbers (zero indexed this time.)  Thus, a list [0, 2, 3] will combine the 0th, 2nd, and 3rd presidents into a group, which is then compared with all presidents not in the list.  You can use get_inlist to generate this list from a cluster number, or you can pick a single president with a single element list like [2].

The function finds the median weight in the "inlist" and the median weight in the "outlist," subtracting the two to find the score for each word.  In the resulting list, a positive score indicates that a word has a high weight in the inlist, while a negative score indicates that it has a high weight in the outlist.  If there are an odd number of presidents in the list, the median takes the lower of the two middle numbers.

Thus, to find the most characteristic words in the inlist, scroll to the end of the output and look at the words with the highest score

In [52]:
import statistics

def medlow(lst):
    sortlst = sorted(lst)
    if len(sortlst) % 2 == 0:
        return(sortlst[int(len(sortlst) / 2) - 1])
    else:
        return(sortlst[int(len(sortlst) / 2 - 0.5)])

def get_inlist(lab, i):
    return [j for j in range(0, len(lab)) if lab[j] == i]
    
def get_characteristic_words(overrep_words, inlist):
    wordscore_in = dict()
    wordscore_out = dict()
    for k in range(0, len(overrep_words)):
        if k in inlist:
            for word, score in overrep_words[k]:
                wordscore_in.setdefault(word, list())
                wordscore_in[word].append(score)
        else:
            for word, score in overrep_words[k]:
                wordscore_out.setdefault(word, list())
                wordscore_out[word].append(score)
    for word in wordscore_in.keys():
        for i in range(0, len(inlist) - len(wordscore_in[word])):
            wordscore_in[word].append(0)
    for word in wordscore_out.keys():
        for i in range(0, (len(overrep_words) - len(inlist)) - len(wordscore_out[word])):
            wordscore_out[word].append(0)
    scorelist = list()
    for word in wordscore_in.keys():
        med_in = medlow(wordscore_in[word]) if word in wordscore_in.keys() else 0
        med_out = medlow(wordscore_out[word]) if word in wordscore_out.keys() else 0
        scorelist.append((word, med_in - med_out))
    return sorted(scorelist, key = lambda x: x[1])

print(get_characteristic_words(overrep_words, get_inlist(clustering.labels_, 0)))

[('constitution', -5.758472952547098), ('peace', -4.572409548710276), ('solemn', -4.344152752217254), ('nation', -3.912158129369412), ('wrongs', -3.7931803482475255), ('union', -3.655176188646093), ('enlightened', -3.4824589753463746), ('hostilities', -3.1639423406200295), ('blessings', -3.1365154850807175), ('vigor', -3.00513801874269), ('intercourse', -2.6494132322253847), ('principles', -2.5361789252945055), ('sentiments', -2.4438992069847747), ('hereafter', -2.2884081828848464), ('commencement', -2.238652566185517), ('gratitude', -2.2277073083927834), ('nations', -2.1746053704410517), ('observance', -2.168067575317249), ('prosperous', -2.1547794040617765), ('wisdom', -1.9981166648061228), ('citizens', -1.9868196956042166), ('millions', -1.957854623469138), ('president', -1.880333780853762), ('negotiation', -1.7305815030841702), ('spirit', -1.705621299392997), ('preserve', -1.6348278410041843), ('occasion', -1.5138526318926238), ('aggression', -1.463767725541869), ('expedient', -1.3

Characteristic words for cluster 0 (Grant through Hoover): speedily, gratifying, continuance, intrusted, tariff, appropriation, furnish, guaranty, enactment, appropriations

Counting only nouns and words with semantic content:  tariff, appropriation, appropriations, navy, receipts, postmaster, fiscal, commissioners, legislation, treasury

Conclusion: There seems to be more talk of financial issues than with the other clusters (tariff, appropriation, receipts, fiscal, treasury).

In [53]:
print(get_characteristic_words(overrep_words, get_inlist(clustering.labels_, 1)))

[('treasury', -14.033228357968794), ('upon', -13.89234970283606), ('appropriations', -13.849596041497854), ('earnestly', -13.751812571630044), ('expenditures', -13.559624133459154), ('appropriation', -11.992943533628123), ('intercourse', -11.961351110467156), ('commerce', -11.594140468902916), ('consideration', -11.516379321125237), ('gratifying', -11.057199985343754), ('expedient', -10.869320395260877), ('practicable', -10.54031947591348), ('duties', -10.173615647474874), ('receipts', -10.066714072847217), ('peculiarly', -9.980044668314605), ('treaty', -9.965085953128453), ('necessity', -9.847012018743259), ('constitution', -9.815842920341105), ('tariff', -9.754452449206607), ('justly', -9.377126473827278), ('furnish', -9.348215813607013), ('provision', -8.955103851118153), ('states', -8.850364146381146), ('evils', -8.803050586710679), ('hereafter', -8.505769679345745), ('favorable', -8.352092954057834), ('satisfactory', -8.082950317594268), ('countrymen', -7.882593832971851), ('duty'

Characteristic words for cluster 1 (FDR through Trump): america's, america, americans, american, peace, bipartisan, allies, nation's, strengthen, nation

Conclusion: this is the "America as a great power" cluster.  America has allies, wants to preserve peace, and has a strong identity as a nation.  There is also some partisan division that leads to valuing "bipartisan" legislation.

In [54]:
print(get_characteristic_words(overrep_words, get_inlist(clustering.labels_, 2)))

[('nation', -9.779741487604674), ('legislation', -6.486128253111186), ('american', -5.821040942143958), ('our', -5.575982135014223), ('abroad', -5.404209538297495), ("nation's", -5.201646454127846), ('allies', -5.087945324999088), ('progress', -4.6857944801272895), ('secure', -4.623095239756847), ('railroads', -4.5859843297892215), ('toward', -4.406189233459911), ('peace', -3.978807519327824), ('america', -3.8744429252050274), ('recommendations', -3.568821494376908), ('renew', -3.512978733694023), ('freedom', -3.3325897543182683), ('burdens', -3.31597749539044), ('revenues', -3.225704159615219), ('cooperation', -3.211938821043497), ('necessities', -3.0991930678694772), ('enacted', -2.741254121014553), ('hopeful', -2.705497925270939), ('strengthen', -2.63637725847227), ('defenses', -2.627764586001367), ('determination', -2.4727276994185496), ('responsibility', -2.4470622657899255), ('accomplished', -2.2115628278399218), ('aggression', -2.147360060077264), ('cooperate', -2.14503120588339

Characteristic words for cluster 2 (Washington through Johnson): intrusted, heretofore, effectually, herewith, effectual, intercourse, salutary, continuance, constitution, expedience

Counting only nouns and words with semantic content: contitution, postmaster, duties, framers, treasury, commerce, vessels, harbors, cargoes, provisions

Presidents are probably talking a lot more formally, and only some people in the country - the elites - can vote.  People are still talking about the recently-devised constitution and its framers.  Trade and commerce are clearly important; perhaps the main function of the federal government at this time is to manage commerce?